In [54]:
%matplotlib inline

import sys
sys.version

import numpy as np
import pystan
import scipy.io as sio
import time

In [103]:
# Load data from matlab
str = '/Users/samlewallen/Documents/neuroscience/BRAIN_data_project/Nbins.mat'
mat_contents = sio.loadmat(str)
Nbins = mat_contents['Nbins']
str = '/Users/samlewallen/Documents/neuroscience/BRAIN_data_project/x.mat'
mat_contents = sio.loadmat(str)
x = mat_contents['x']
str = '/Users/samlewallen/Documents/neuroscience/BRAIN_data_project/y.mat'
mat_contents = sio.loadmat(str)
y = mat_contents['y']
x = np.float64(x)
y = np.float64(y)
Nbins = np.int(Nbins)
x = x.transpose()
y = y.transpose()
x = x.squeeze()
y = y.squeeze()

In [105]:
t = time.time()
ocode = """
   data {
       int<lower=1> Nbins; // number of bins 
       vector[Nbins] x; // position bins 
       vector[Nbins] y; // data 
   }
   transformed data {
       int<lower=1> Nbumps; // number of bumps to consider % for now, hard-coded   
        real<lower=0> sigmaSqGP;
        real<lower=0> etaSqGP;
        real<lower=0> rhoSqGP;
        Nbumps = 5; 
        sigmaSqGP = .01;
        etaSqGP = .01;
        rhoSqGP = 1;
   }
   parameters {
       simplex[Nbumps] lambda; // scaled "bump domains" 
       vector<lower=0>[Nbumps] bumpSigma; // bump widths 
       vector<lower=0>[Nbumps] bumpHeight; // bump heights
       // real<lower=0> sigmaSqGP; 
       // real<lower=0> etaSqGP; 
       // real<lower=0> rhoSqGP; 
   }  
   transformed parameters {
       vector<lower=x[1],upper=x[Nbins]>[Nbumps] bumpCtrs; // bump centers 
       vector<lower=x[1],upper=x[Nbins]>[Nbumps] bumpDomainLengths; 
       bumpDomainLengths = (x[Nbins] - x[1])*lambda;  
       bumpCtrs[1] = .5*bumpDomainLengths[1];    
       for (n in 2:Nbumps)
         bumpCtrs[n] = bumpCtrs[n-1] + .5*bumpDomainLengths[n-1] + .5*bumpDomainLengths[n]; 
   }
   model {         
       vector[Nbins] mu; 
       matrix[Nbins,Nbins] Sigma; 
       vector[Nbumps] alpha; 
       for (i in 1:Nbumps) 
         alpha[i] = 3;
   
      // compute GP mean mu: 
       mu = 0*y;   
       for (n in 1:Nbumps)
         mu = mu + bumpHeight[n] * exp( -( x-bumpCtrs[n]).*(x-bumpCtrs[n] ) / (2*bumpSigma[n]^2) ); 
   
      // compute GP cov Sigma:    
      // off-diagonal elements 
       for (i in 1:(Nbins-1)) { 
         for (j in (i+1):Nbins) { 
           Sigma[i, j] = etaSqGP * exp(-rhoSqGP * pow(x[i] - x[j],2)); 
           Sigma[j, i] = Sigma[i, j]; 
          } 
       } 
      // diagonal elements 
       for (k in 1:Nbins)  
           Sigma[k, k] = etaSqGP + sigmaSqGP;  // + jitter 
   
      // models and priors: 
       lambda ~ dirichlet(alpha); 
       for (i in 1:Nbumps) 
           bumpSigma[i] ~ normal(8,3); 
       for (i in 1:Nbumps) 
           bumpHeight[i] ~ normal(1,.3); 
       // etaSqGP ~ cauchy(0, .1); 
       // rhoSqGP ~ cauchy(0, 1); 
       // sigmaSqGP ~ cauchy(0, .1); 
   
      // process: 
       y ~ multi_normal(mu, Sigma); 
   }
"""
sm = pystan.StanModel(model_code=ocode)
elapsed = time.time() - t
print(elapsed)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_5d1ec03d0e40de3dc4d5c71af6f4d73a NOW.


30.511999130249023


In [68]:
datain = {'x': x,
        'y': y,
        'Nbins': Nbins
       }
op = sm.optimizing(data=datain)
str = '/Users/samlewallen/Documents/neuroscience/BRAIN_data_project/map.mat'
sio.savemat(str,op)

In [109]:
t = time.time()
fit = sm.sampling(data=datain,iter=2000, chains=2)
elapsed = time.time() - t
str = '/Users/samlewallen/Documents/neuroscience/BRAIN_data_project/fit.mat'
sio.savemat(str,fit.extract())
print(elapsed)
fit

36.807563066482544


Inference for Stan model: anon_model_5d1ec03d0e40de3dc4d5c71af6f4d73a.
2 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=2000.

                       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
lambda[0]              0.26    0.15   0.15   0.06   0.11   0.28   0.41   0.42      1   9.98
lambda[1]              0.24    0.05   0.05   0.13   0.19   0.26   0.27    0.3      1   1.57
lambda[2]              0.29    0.19   0.19   0.06    0.1   0.26   0.48   0.55      1   6.35
lambda[3]              0.13  4.9e-3   0.04   0.05   0.11   0.13   0.15    0.2     60   1.04
lambda[4]               0.1  2.5e-3   0.02   0.06   0.09    0.1   0.11   0.14     70   1.03
bumpSigma[0]           6.96    3.84   3.84   0.51   2.82   8.07   9.97  13.32      1   1.63
bumpSigma[1]          11.09    1.44   2.04   7.27   9.55  11.16  12.53  14.98      2   1.35
bumpSigma[2]           7.47    5.55   5.55   0.16   1.48   9.88  12.58  15.

In [108]:
fit.extract('bumpHeight')

OrderedDict([('bumpHeight',
              array([[ 0.93003183,  1.06907211,  1.33177389,  0.33392324,  0.31640749],
                     [ 0.93785883,  1.20738098,  0.67972548,  0.61116771,  0.76595339],
                     [ 0.80772965,  1.10899196,  0.8953727 ,  0.40598797,  1.38987105],
                     ..., 
                     [ 1.49481265,  0.81634302,  1.05731252,  0.30569385,  0.54227119],
                     [ 1.4696026 ,  0.80443629,  1.22259213,  0.45043697,  0.86776437],
                     [ 0.85852891,  0.7521639 ,  1.06925778,  0.17787774,  1.07541832]]))])

In [98]:
fit

Inference for Stan model: anon_model_708d9d0d02e5020b135942ec27793dbf.
1 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=500.

                       mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
lambda[0]              0.05  2.2e-3   0.01   0.03   0.05   0.06   0.06   0.07     25   1.03
lambda[1]              0.03  2.4e-3   0.01   0.01   0.02   0.03   0.04   0.07     27   1.05
lambda[2]              0.91  4.7e-4 5.3e-3    0.9   0.91   0.91   0.92   0.92    130   1.02
bumpSigma[0]           2.51    0.41   2.52   0.35   1.06    1.7   2.53  10.35     38   1.01
bumpSigma[1]           3.95    0.75   3.51   0.03   1.34   2.79   5.91  12.66     22   1.11
bumpSigma[2]          12.62    0.16   1.57   9.57  11.41  12.57  13.82  15.69     96   1.02
bumpHeight[0]           0.8    0.05   0.39   0.14   0.49   0.84    1.1   1.47     55   1.01
bumpHeight[1]          0.73    0.06   0.38   0.17   0.38   0.75   1.02   1.44 